Enunciado
Desarrollá un script o notebook que realice lo siguiente:
1. Leer el archivo.
2. Identificar el día de la semana de cada entrenamiento.
3. Indicar cuál es el/los día/s de la semana que tienen más sesiones de entrenamiento.
4. Cuántos días pasaron entre el primer y último entrenamiento.
5. Mostrar el campeón que más entrenó.
6. Promedio de entrenamientos por cada día de la semana.
7. Campeón que más entrena los fines de semana (sábado y domingo).
8. En una carpeta llamada salida, que se debe generar, crear un archivo .csv que para
cada campeón muestre la cantidad de entrenamientos (indicar campeón y cantidad).
9. Generar un archivo .json en la misma carpeta (salida) que indique el total de
registros y para cada día, los campeones que entrenaron, junto con cuántas veces
entrenó cada uno.

*Idealmente recorrer el archivo 1 sola vez // tip de la profe: generar una funcion por cada row que se llaman desde el programa principal.*

In [1]:
#imports
from pathlib import Path
from datetime import datetime
import csv
import json
import os

#configuracion.
cwd = Path(".").resolve()
route = Path(cwd, "actividad_2.csv")

#variables principales
contador_dia = {}
primera_fecha = None
ultima_fecha = None
campeones = {}
detalle_dias = {}
total_registros = 0


In [2]:
#funciones

#Identificar el día de la semana
def obtener_dia(fecha_texto):
    fecha = datetime.strptime(fecha_texto, "%Y-%m-%d %H:%M")
    dia = fecha.strftime("%A")
    return fecha, dia


#Contar entrenamientos por día
def contar_dias(dia_semana, contador_dia):
    if dia_semana in contador_dia:
        contador_dia[dia_semana] += 1
    else:
        contador_dia[dia_semana] = 1


#Contar entrenamientos por campeón
def contar_campeon(campeon, campeones):
    if campeon not in campeones:
        campeones[campeon] = {"total": 0, "fin_de_semana": 0}
    campeones[campeon]["total"] += 1


#Si es fin de semana, contar aparte
def contar_finde(dia_semana, campeon, campeones):
    if dia_semana == "Saturday" or dia_semana == "Sunday":
        campeones[campeon]["fin_de_semana"] += 1


#Actualizar fechas mínima y máxima
def actualizar_fechas(fecha, primera_fecha, ultima_fecha):
    if primera_fecha is None or fecha < primera_fecha:
        primera_fecha = fecha
    if ultima_fecha is None or fecha > ultima_fecha:
        ultima_fecha = fecha
    return primera_fecha, ultima_fecha


# 6. Guardar info para el JSON
def actualizar_detalle(dia_semana, campeon, detalle_dias):
    if dia_semana not in detalle_dias:
        detalle_dias[dia_semana] = {}
    if campeon not in detalle_dias[dia_semana]:
        detalle_dias[dia_semana][campeon] = 1
    else:
        detalle_dias[dia_semana][campeon] += 1


# 7. Día/s con más entrenamientos
def dias_max_sesiones(contador_dia):
    maximo = max(contador_dia.values())
    dias = []
    for dia in contador_dia:
        if contador_dia[dia] == maximo:
            dias.append(dia)
    return dias, maximo


# 8. Campeón que más entrenó
def campeon_max(campeones):
    max_entrenamientos = 0
    nombre = ""
    for campeon in campeones:
        if campeones[campeon]["total"] > max_entrenamientos:
            max_entrenamientos = campeones[campeon]["total"]
            nombre = campeon
    return nombre, max_entrenamientos


# 9. Campeón que más entrena fines de semana
def campeon_max_finde(campeones):
    max_finde = 0
    nombre = ""
    for campeon in campeones:
        if campeones[campeon]["fin_de_semana"] > max_finde:
            max_finde = campeones[campeon]["fin_de_semana"]
            nombre = campeon
    return nombre, max_finde


# 10. Promedio de entrenamientos por día
def promedio_dias(contador_dia):
    total = sum(contador_dia.values())
    promedios = {}
    for dia in contador_dia:
        promedios[dia] = round(contador_dia[dia] / total, 3)
    return promedios


# 11. Crear CSV en carpeta salida
def generar_csv(campeones):
    os.makedirs("salida", exist_ok=True)
    with open("salida/entrenamientos_por_campeon.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Campeon", "Cantidad"])
        for campeon in campeones:
            writer.writerow([campeon, campeones[campeon]["total"]])


# 12. Crear JSON en carpeta salida
def generar_json(total, detalle_dias):
    os.makedirs("salida", exist_ok=True)
    datos = {
        "total_registros": total,
        "detalle_dias": detalle_dias
    }
    with open("salida/resumen.json", "w", encoding="utf-8") as f:
        json.dump(datos, f, indent=4, ensure_ascii=False)



In [5]:
#programa principal

with open(route, "r", encoding="utf-8") as archivo:
    reader = csv.DictReader(archivo)

    for row in reader:
        total_registros += 1
        fecha, dia_semana = obtener_dia(row["timestamp"])
        campeon = row["campeon"]
        contar_dias(dia_semana, contador_dia)
        contar_campeon(campeon, campeones)
        contar_finde(dia_semana, campeon, campeones)
        primera_fecha, ultima_fecha = actualizar_fechas(fecha, primera_fecha, ultima_fecha)
        actualizar_detalle(dia_semana, campeon, detalle_dias)


#Prints

dias_max, cantidad_max = dias_max_sesiones(contador_dia)
print("Día/s con más entrenamientos:", dias_max, "(", cantidad_max, ")")

dias_diferencia = (ultima_fecha - primera_fecha).days
print("Días entre primer y último entrenamiento:", dias_diferencia)

campeon_top, entrenos_top = campeon_max(campeones)
print("Campeón que más entrenó:", campeon_top, "(", entrenos_top, ")")

print("Promedio de entrenamientos por día:", promedio_dias(contador_dia))

campeon_finde, cant_finde = campeon_max_finde(campeones)
print("Campeon con mas entrenamientos en el fin de semana:", campeon_finde, "(", cant_finde, ")")


#Generador de archivos

generar_csv(campeones)
generar_json(total_registros, detalle_dias)

print("Archivos generados en carpeta 'salida'")


Día/s con más entrenamientos: ['Monday', 'Tuesday', 'Wednesday', 'Thursday'] ( 176 )
Días entre primer y último entrenamiento: 74
Campeón que más entrenó: Vi ( 126 )
Promedio de entrenamientos por día: {'Sunday': 0.142, 'Monday': 0.147, 'Tuesday': 0.147, 'Wednesday': 0.147, 'Thursday': 0.147, 'Friday': 0.138, 'Saturday': 0.133}
Campeón con más entrenamientos en fin de semana: Garen ( 40 )

✅ Archivos generados en carpeta 'salida'


In [6]:
!git config --global user.name "donguiboy"
!git config --global user.email "gonzaloflara@gmail.com"


In [7]:
!git init
!git remote add origin https://github.com/donguiboy/actividad_2_lenguajes.git
!git branch -M main


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
